In [2]:
#As always, we import everything
import os
import pandas as pd
import json
import folium
import math

First, let's get the data and put it in a dataframe.

In [3]:
raw_eu_data = pd.read_table('tsdec450.tsv')

Now, we can process it to get only the value we're interested in, and to get the id for each country.

In [4]:
eu_data = raw_eu_data[[name.split(",")[1] == 'PC_ACT' for name in raw_eu_data['info']]]
eu_data['info'] = eu_data['info'].apply(lambda x: x.split(",")[3])
eu_data

/usr/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,info,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,AT,:,:,:,:,4.3,4.2,4.7,4.7,4.7,...,4.9,4.1,5.3,4.8,4.6,4.9,5.4,5.6,5.7,6.0
1,BE,6.6,6.4,7.1,8.6,9.8,9.7,9.5,9.2,9.3,...,7.5,7,7.9,8.3,7.2,7.6,8.4,8.5,8.5,7.8
2,BG,:,:,:,:,:,:,:,:,:,...,6.9,5.6,6.8,10.3 i,11.3,12.3,13.0,11.4,9.2,7.6
3,CY,:,:,:,:,:,:,:,:,:,...,3.9,3.7,5.4,6.3,7.9,11.9,15.9,16.1,15.0,13.0
4,CZ,:,:,:,4.3,4.3,4,3.9,4.8,6.5,...,5.3,4.4,6.7,7.3,6.7,7.0,7.0,6.1,5.1,4.0
5,DE,:,5.5,6.6,7.8,8.4,8.2,8.9,9.6,9.4,...,8.5,7.4,7.6,7,5.8,5.4,5.2,5.0,4.6,4.1
6,DK,7.2,7.9,8.6,9.6,7.7,6.7,6.3,5.2,4.9,...,3.8,3.4,6,7.5,7.6,7.5,7.0,6.6,6.2,6.2
7,EA18,:,:,:,:,:,:,:,:,:,...,7.5,7.6,9.6,10.1,10.1,11.4,12.0,11.6,10.9,10.0
8,EA19,:,:,:,:,:,:,:,:,:,...,7.5,7.6,9.6,10.2,10.2,11.4,12.0,11.6,10.9,10.0
9,EE,:,:,:,:,:,:,:,:,:,...,4.6,5.5 i,13.5,16.7,12.3,10.0,8.6,7.4,6.2,6.8


Now we can define a function that returns the color of a country based on its unemployement rate.
For this, we need to choose a palette of colors : we used colorbrewer to get one which is color-blind friendly.
To get the color for a country, we simply bin all the unemployement_rate into one of the colors.

In [5]:
#colors_unemployement = ['#fee5d9','#fcbba1','#fc9272','#fb6a4a','#de2d26','#a50f15']
colors_unemployement = ['#fef0d9','#fdd49e','#fdbb84','#fc8d59','#e34a33','#b30000']
min_unemployement_eu = eu_data['2016'].min()
max_unemployement_eu = eu_data['2016'].max()
def get_color_eu(country):
    values = eu_data.loc[eu_data['info'] == country, '2016'].values
    if len(values) == 0:
        return '#000000'
    
    unemployement_rate = values[0]
    ratio = (unemployement_rate - min_unemployement_eu) / (max_unemployement_eu - min_unemployement_eu)
    index = math.floor(ratio * len(colors_unemployement))
    return colors_unemployement[index]

#TODO : Check colors + maybe change intervals

We also define a function that checks if a country is in the dataframe, and if not, we make its overly transparent.

In [6]:
def get_opacity_eu(country):
    values = eu_data.loc[eu_data['info'] == country, '2016'].values
    return 1 if len(values) > 0 else 0

Now we can create the Europe map, and add the overlay, using the previous functions.

We used to following website to convert the topojson file to gejson
https://jeffpaine.github.io/geojson-topojson/

In [9]:
map_eu = folium.Map([51,15], tiles='cartodbpositron', zoom_start=4)


# Color of the country
folium.TopoJson(
    open('topojson/europe.topojson.json'),
    object_path='objects.europe',
    style_function=lambda feature: {
        'fillOpacity' : get_opacity_eu(feature['id']), #opacity for the fill color
        'opacity' : get_opacity_eu(feature['id']), #opacity for the borders
        'fillColor': get_color_eu(feature['id']),
        'color' : 'black',
        'weight' : 1
        }
).add_to(map_eu)

# 
map_eu.choropleth(geo_data='topojson/europe.geojson.json',data=eu_data,
             columns=['info', '2016'],
             key_on='feature.id',
             fill_color='OrRd',
             #fill_color='RdGy',
             fill_opacity=0, 
             line_opacity=0.0,
             legend_name='Percentage of unemployement in country')

map_eu

In [13]:
raw_switzerland = pd.read_table('data/unemployed_switzerland.csv')

In [14]:
raw_switzerland

,",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"
0,",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"
1,"Canton,Sexe,NOGA 08 à 2 chiffres,,Mois,Janvier..."
2,", , ,,Mesures,Chômeurs inscrits,Chômeurs jeun..."
3,"Zurich,Hommes,1,""Culture et production animale..."
4,"Zurich,Hommes,2,Sylviculture et exploitation f..."
5,"Zurich,Hommes,3,Pêche et aquaculture,,0,0,0,0,..."
6,"Zurich,Hommes,5,Extraction de houille et de li..."
7,"Zurich,Hommes,7,Extraction de minerais métalli..."
8,"Zurich,Hommes,8,Autres industries extractives,..."
9,"Zurich,Hommes,9,Services de soutien aux indust..."
